# DreamBooth 实现

这个 Notebook 实现了 DreamBooth 算法，用于微调 Stable Diffusion 模型以学习新的概念（例如特定对象或风格）。

它包含以下部分：
1.  **依赖项检查和导入**
2.  **辅助函数定义** (数据集加载、稀有令牌查找、类别图像生成等)
3.  **训练函数** (`dreambooth_training`)
4.  **推理函数** (`inference`)
5.  **配置和执行** (设置参数并运行训练或推理)

## 1. 依赖项检查和导入

In [2]:
import os
import random
import numpy as np
import argparse
from pathlib import Path
from tqdm.auto import tqdm
from PIL import Image

# 首先检查依赖项
def check_dependencies():
    missing_deps = []
    try:
        import torch
    except ImportError:
        missing_deps.append("torch")
    
    try:
        import accelerate
    except ImportError:
        missing_deps.append("accelerate")
        
    try:
        import transformers
    except ImportError:
        missing_deps.append("transformers")
        
    try:
        import diffusers
    except ImportError:
        missing_deps.append("diffusers")
    
    return missing_deps

missing = check_dependencies()
if missing:
    print("缺少必要的依赖项，请先安装：")
    print(f"pip install {' '.join(missing)}")
    print("\n如果遇到Flash Attention相关错误，请尝试：")
    print("pip install diffusers==0.19.3 transformers==4.30.2 accelerate xformers")
    # 在Notebook中，我们通常不直接退出，而是抛出错误或显示消息
    raise ImportError(f"缺少依赖: {', '.join(missing)}")
else:
    print("所有基本依赖项已满足。")

# 正常导入，现在有更好的错误处理
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

try:
    # 尝试导入主要模块
    from diffusers import StableDiffusionPipeline, DDPMScheduler, UNet2DConditionModel
    from transformers import CLIPTextModel, CLIPTokenizer
    print("Diffusers 和 Transformers 导入成功。")
except ImportError as e:
    print(f"导入错误: {e}")
    print("\n请尝试安装兼容版本的依赖项:")
    print("pip install diffusers==0.19.3 transformers==4.30.2 accelerate")
    print("如果您使用NVIDIA GPU，可以添加: xformers")
    raise e

from accelerate import Accelerator
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")

c:\Users\xy\miniconda3\envs\dreambooth\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


所有基本依赖项已满足。
Diffusers 和 Transformers 导入成功。
PyTorch 版本: 2.7.0+cu128
CUDA 可用: True


## 2. 辅助函数定义

In [3]:
class DreamBoothDataset(Dataset):
    def __init__(self, instance_images_path, class_images_path, tokenizer, size=512, center_crop=True):
        self.size = size
        self.center_crop = center_crop
        self.tokenizer = tokenizer
        
        # 简化图像加载
        self.instance_images = self._load_images(instance_images_path) if os.path.exists(instance_images_path) else []
        self.class_images = self._load_images(class_images_path) if class_images_path and os.path.exists(class_images_path) else []
        print(f"加载了 {len(self.instance_images)} 张实例图像和 {len(self.class_images)} 张类别图像")
    
    def _load_images(self, path):
        images = []
        if not path:
            return images
            
        print(f"从 {path} 加载图像...")
        for file in tqdm(os.listdir(path), desc=f"加载 {os.path.basename(path)}"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    img = Image.open(os.path.join(path, file)).convert('RGB')
                    # 简化图像处理
                    if self.center_crop:
                        w, h = img.size
                        min_dim = min(w, h)
                        img = img.crop(((w-min_dim)//2, (h-min_dim)//2, (w+min_dim)//2, (h+min_dim)//2))
                    img = img.resize((self.size, self.size))
                    images.append(img)
                except Exception as e:
                    print(f"警告: 跳过无法加载的图像 {file}: {e}")
        return images
    
    def __len__(self):
        # 确保至少有一个实例图像
        if not self.instance_images:
             raise ValueError("错误: 实例图像目录为空或无法加载任何图像。请检查 instance_data_dir。")
        return max(len(self.instance_images), len(self.class_images))
    
    def __getitem__(self, idx):
        item = {}
        # 获取实例图像 (循环使用如果数量少于类别图像)
        instance_image = self.instance_images[idx % len(self.instance_images)]
        instance_image_tensor = torch.from_numpy(np.array(instance_image).astype(np.float32) / 127.5 - 1.0)
        item["instance_pixel_values"] = instance_image_tensor.permute(2, 0, 1)
        
        # 获取类别图像 (如果存在)
        if self.class_images:
            class_image = self.class_images[idx % len(self.class_images)]
            class_image_tensor = torch.from_numpy(np.array(class_image).astype(np.float32) / 127.5 - 1.0)
            item["class_pixel_values"] = class_image_tensor.permute(2, 0, 1)
            
        return item

def find_rare_token(tokenizer, token_range=(5000, 10000)):
    """
    按照论文所述，查找稀有令牌作为标识符
    对于Stable Diffusion，我们使用CLIP tokenizer
    """
    while True:
        token_id = random.randint(token_range[0], token_range[1])
        # 确保选择的是符合条件的token（3个或更少Unicode字符，不含空格）
        token_text = tokenizer.decode([token_id]).strip()
        if 1 <= len(token_text) <= 3 and ' ' not in token_text and token_text.isprintable():
             # 进一步检查是否是特殊标记或常用词的一部分 (简单检查)
            if not token_text.startswith('<') and not token_text.endswith('>') and len(token_text) > 1:
                 return token_text

def generate_class_images(model, class_prompt, output_dir, num_samples=200):
    """
    生成类别图像以用于先验保留损失
    这是论文中提到的先验保留机制的关键部分
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    else:
        # 如果目录已存在且包含足够图片，则跳过生成
        existing_files = [f for f in os.listdir(output_dir) if f.lower().endswith('.png')]
        if len(existing_files) >= num_samples:
            print(f"已在 {output_dir} 找到 {len(existing_files)} 张类别图像，跳过生成。")
            return output_dir
        else:
             print(f"在 {output_dir} 找到 {len(existing_files)} 张类别图像，需要生成 {num_samples - len(existing_files)} 张。")
             num_samples -= len(existing_files)
             start_idx = len(existing_files)

    print(f"正在生成 {num_samples} 张类别图像用于先验保留到 {output_dir}...")
    
    # 将模型设置为推理模式
    model.safety_checker = None  # 禁用安全检查器以加快生成
    device = model.device
    
    # 批量生成图像以加快速度
    batch_size = 8 # 增加批量大小以提高效率
    num_batches = (num_samples + batch_size - 1) // batch_size
    
    generated_count = 0
    for batch_idx in tqdm(range(num_batches), desc="生成类别图像"):
        current_batch_size = min(batch_size, num_samples - generated_count)
        if current_batch_size <= 0:
            break
        batch_prompts = [class_prompt] * current_batch_size
        with torch.no_grad():
            # 确保模型在正确的设备上
            if model.device != device:
                 model.to(device)
            outputs = model(batch_prompts, num_inference_steps=50, guidance_scale=7.5)
            
        for i, image in enumerate(outputs.images):
            img_idx = start_idx + generated_count + i
            image.save(os.path.join(output_dir, f"class_{img_idx:04d}.png"))
        generated_count += len(outputs.images)
            
    print(f"成功生成 {generated_count} 张类别图像。")
    return output_dir

def download_small_model():
    """
    自动下载小型模型，适合低资源设备
    返回模型的路径或名称
    """
    print("选择适合低资源设备的小型模型...")
    
    # 定义小型模型选项 - 更新为较旧但稳定且兼容性更好的版本
    small_models = [
        "CompVis/stable-diffusion-v1-4",                # 较旧但稳定的SD1.4，兼容性好
        "runwayml/stable-diffusion-v1-5",               # 标准SD1.5
        "stabilityai/stable-diffusion-2-base",          # SD2基础版
    ]
    
    # 选择默认小型模型
    chosen_model = small_models[0]
    
    print(f"已选择模型: {chosen_model}")
    print(f"此模型与较旧版本的diffusers兼容性更好")
    print("模型将在首次使用时自动从Hugging Face下载")
    
    return chosen_model

# 添加这个函数来显示简洁的使用说明
def show_quick_help():
    """显示简洁的使用说明"""
    print("\n请在下方的配置单元格中设置参数并运行后续单元格。")
    print("主要参数:")
    print("  train_mode = True  # 或 False")
    print("  infer_mode = True  # 或 False")
    print("  instance_data_dir = './my_images' # 包含你的训练图片的目录")
    print("  class_prompt = 'a photo of a dog' # 你的训练对象所属的类别")
    print("  output_dir = './output' # 模型输出目录")
    print("  prompt = 'a photo of a sks dog on the beach' # 推理时使用的提示词")

## 3. 训练函数 (`dreambooth_training`)

In [4]:
def dreambooth_training(
    pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5",
    instance_data_dir="./instance_images",
    output_dir="./output",
    class_prompt="a dog",
    instance_prompt_template="a photo of a {} dog", # 使用模板，{} 会被稀有令牌替换
    learning_rate=5e-6,
    max_train_steps=800, # 减少默认步数以加快示例运行
    prior_preservation_weight=1.0,  # λ参数，控制先验保留损失的权重
    prior_generation_samples=50, # 减少默认类别图像数量以节省时间和资源
    gradient_accumulation_steps=1,
    train_text_encoder=True,  # 是否微调文本编码器
    train_batch_size=1,
    seed=42,
    mixed_precision="fp16" # 默认启用混合精度
):
    # 设置种子以保证结果可复现
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    
    # 初始化加速器
    accelerator = Accelerator(
        gradient_accumulation_steps=gradient_accumulation_steps,
        mixed_precision=mixed_precision
    )
    
    # 初始化tokenizer
    try:
        tokenizer = CLIPTokenizer.from_pretrained(
            pretrained_model_name_or_path, 
            subfolder="tokenizer"
        )
    except Exception as e:
         print(f"加载 Tokenizer 失败: {e}")
         print(f"尝试不指定 subfolder 加载...")
         tokenizer = CLIPTokenizer.from_pretrained(pretrained_model_name_or_path)
    
    # 查找稀有令牌作为标识符，如论文3.2节所述
    identifier = find_rare_token(tokenizer)
    print(f"选中的稀有令牌标识符: '{identifier}'")
    
    # 构建实例提示词
    instance_prompt = instance_prompt_template.format(identifier)
    
    print(f"实例提示词: '{instance_prompt}'")
    print(f"类别提示词: '{class_prompt}'")
    
    # 加载预训练模型
    print(f"加载预训练模型: {pretrained_model_name_or_path}")
    try:
        # 加载文本编码器
        text_encoder = CLIPTextModel.from_pretrained(
            pretrained_model_name_or_path, 
            subfolder="text_encoder"
        )
        # 加载U-Net
        unet = UNet2DConditionModel.from_pretrained(
            pretrained_model_name_or_path,
            subfolder="unet",
        )
        # 加载用于生成类别图像的完整pipeline
        pipeline_for_gen = StableDiffusionPipeline.from_pretrained(
            pretrained_model_name_or_path,
        )
    except Exception as e:
         print(f"加载模型组件失败: {e}")
         print(f"尝试不指定 subfolder 加载...")
         text_encoder = CLIPTextModel.from_pretrained(pretrained_model_name_or_path)
         unet = UNet2DConditionModel.from_pretrained(pretrained_model_name_or_path)
         pipeline_for_gen = StableDiffusionPipeline.from_pretrained(pretrained_model_name_or_path)

    pipeline_for_gen.to(accelerator.device)
    
    # 生成类别图像用于先验保留
    class_images_dir = os.path.join(output_dir, "class_images")
    generate_class_images(pipeline_for_gen, class_prompt, class_images_dir, prior_generation_samples)
    # 释放生成模型的显存
    del pipeline_for_gen
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # 加载数据集
    dataset = DreamBoothDataset(
        instance_images_path=instance_data_dir,
        class_images_path=class_images_dir,
        tokenizer=tokenizer,
    )
    dataloader = DataLoader(dataset, batch_size=train_batch_size, shuffle=True, num_workers=0) # num_workers=0 避免多进程问题
    
    # 准备优化器
    params_to_optimize = list(unet.parameters())
    if train_text_encoder:
        print("将同时微调U-Net和文本编码器")
        params_to_optimize += list(text_encoder.parameters())
        # 确保文本编码器需要梯度
        text_encoder.requires_grad_(True)
    else:
        print("仅微调U-Net，冻结文本编码器")
        text_encoder.requires_grad_(False)
    
    optimizer = torch.optim.AdamW(
        params_to_optimize,
        lr=learning_rate,
        betas=(0.9, 0.999),
        weight_decay=1e-2,
    )
    
    # 准备噪声调度器
    try:
        noise_scheduler = DDPMScheduler.from_pretrained(
            pretrained_model_name_or_path, 
            subfolder="scheduler"
        )
    except Exception as e:
         print(f"加载 Scheduler 失败: {e}")
         print(f"尝试不指定 subfolder 加载...")
         noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path)
    
    # 准备文本嵌入 (移到训练循环内部，因为文本编码器可能被训练)
    # instance_text_inputs = tokenizer(...)
    # class_text_inputs = tokenizer(...)
    
    # 将模型、优化器和数据加载器准备用于加速训练
    if train_text_encoder:
        unet, text_encoder, optimizer, dataloader = accelerator.prepare(
            unet, text_encoder, optimizer, dataloader
        )
    else:
         # 如果不训练文本编码器，则不需要 prepare 它
        unet, optimizer, dataloader = accelerator.prepare(
            unet, optimizer, dataloader
        )
        # 将文本编码器移到GPU
        text_encoder.to(accelerator.device)
    
    # 训练循环
    print("开始训练...")
    progress_bar = tqdm(range(max_train_steps), desc="训练进度", disable=not accelerator.is_local_main_process)
    global_step = 0
    
    # 计算文本嵌入的函数
    def get_text_embeddings(prompt_text):
        text_inputs = tokenizer(
            prompt_text,
            padding="max_length",
            max_length=tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt"
        ).input_ids.to(accelerator.device)
        return text_encoder(text_inputs)[0]

    # 使用 accelerator.main_process_first 确保主进程先下载/缓存模型
    with accelerator.main_process_first():
        # 预计算类别提示词嵌入 (如果文本编码器不训练)
        if not train_text_encoder:
            with torch.no_grad():
                precomputed_class_embeds = get_text_embeddings(class_prompt)
        else:
            precomputed_class_embeds = None

    while global_step < max_train_steps:
        unet.train()
        if train_text_encoder:
            text_encoder.train()
        
        for step, batch in enumerate(dataloader):
            if global_step >= max_train_steps:
                break
                
            with accelerator.accumulate(unet):
                # 准备输入
                instance_pixel_values = batch["instance_pixel_values"].to(accelerator.device)
                batch_size = instance_pixel_values.shape[0]
                
                # 获取文本嵌入
                with torch.set_grad_enabled(train_text_encoder):
                    encoder_hidden_states_instance = get_text_embeddings(instance_prompt)
                    if train_text_encoder:
                        encoder_hidden_states_class = get_text_embeddings(class_prompt)
                    else:
                        # 使用预计算的嵌入
                        encoder_hidden_states_class = precomputed_class_embeds
                
                # 为实例图像添加噪声
                noise_instance = torch.randn_like(instance_pixel_values)
                timesteps_instance = torch.randint(
                    0, 
                    noise_scheduler.config.num_train_timesteps,
                    (batch_size,), 
                    device=accelerator.device
                ).long()
                noisy_images_instance = noise_scheduler.add_noise(instance_pixel_values, noise_instance, timesteps_instance)
                
                # 预测实例噪声残差
                noise_pred_instance = unet(noisy_images_instance, timesteps_instance, encoder_hidden_states_instance).sample
                instance_loss = F.mse_loss(noise_pred_instance.float(), noise_instance.float(), reduction="mean")
                
                # 处理类别样本（先验保留）
                class_loss = torch.tensor(0.0, device=accelerator.device, dtype=instance_loss.dtype)
                if "class_pixel_values" in batch and prior_preservation_weight > 0:
                    class_pixel_values = batch["class_pixel_values"].to(accelerator.device)
                    noise_class = torch.randn_like(class_pixel_values)
                    timesteps_class = torch.randint(
                        0, 
                        noise_scheduler.config.num_train_timesteps,
                        (batch_size,), 
                        device=accelerator.device
                    ).long()
                    noisy_images_class = noise_scheduler.add_noise(class_pixel_values, noise_class, timesteps_class)
                    
                    # 预测类别噪声残差
                    noise_pred_class = unet(noisy_images_class, timesteps_class, encoder_hidden_states_class).sample
                    class_loss = F.mse_loss(noise_pred_class.float(), noise_class.float(), reduction="mean")
                
                # 组合损失
                loss = instance_loss + prior_preservation_weight * class_loss
                
                # 反向传播
                accelerator.backward(loss)
                
                # 梯度裁剪 (可选但推荐)
                if accelerator.sync_gradients:
                    params_to_clip = list(unet.parameters())
                    if train_text_encoder:
                        params_to_clip += list(text_encoder.parameters())
                    accelerator.clip_grad_norm_(params_to_clip, 1.0)
                
                # 优化步骤
                optimizer.step()
                optimizer.zero_grad()
                
            # 检查是否需要同步梯度
            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1
                
            # 记录和打印进度
            logs = {"loss": loss.detach().item(), "instance_loss": instance_loss.detach().item(), "class_loss": class_loss.detach().item()}
            progress_bar.set_postfix(**logs)
            # accelerator.log(logs, step=global_step) # 如果配置了 tracker
            
    # 等待所有进程完成
    accelerator.wait_for_everyone()
    
    # 保存微调后的模型 (仅在主进程执行)
    if accelerator.is_main_process:
        unet = accelerator.unwrap_model(unet)
        if train_text_encoder:
            text_encoder = accelerator.unwrap_model(text_encoder)
        
        # 创建输出目录
        os.makedirs(output_dir, exist_ok=True)
        
        # 从原始pipeline创建新的pipeline
        # 需要重新加载原始模型以获取 vae 和 scheduler 等其他组件
        try:
            pipeline = StableDiffusionPipeline.from_pretrained(
                pretrained_model_name_or_path,
                unet=unet,
                text_encoder=text_encoder,
                tokenizer=tokenizer, # 包含 tokenizer
            )
        except Exception as e:
             print(f"从原始模型创建 Pipeline 失败: {e}")
             print(f"尝试不指定 subfolder 加载...")
             pipeline = StableDiffusionPipeline.from_pretrained(
                 pretrained_model_name_or_path,
                 unet=unet,
                 text_encoder=text_encoder,
                 tokenizer=tokenizer,
             )
        
        # 保存微调后的模型
        pipeline.save_pretrained(output_dir)
        print(f"模型已保存到 {output_dir}")
        
        # 保存标识符以供将来推理使用
        with open(os.path.join(output_dir, "identifier.txt"), "w") as f:
            f.write(identifier)
        print(f"标识符 '{identifier}' 已保存到 {os.path.join(output_dir, 'identifier.txt')}")
    
    # 清理显存
    del unet, text_encoder, optimizer, dataloader, dataset, accelerator
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    # 返回标识符，因为 pipeline 对象可能很大且已释放
    return identifier

## 4. 推理函数 (`inference`)

In [5]:
def inference(
    model_path="./output",
    prompt_template="a photo of a {} dog wearing a hat", # 推理提示词模板
    class_prompt="a dog", # 用于提取类别名词
    identifier=None,
    output_image_path="./generated_image.png",
    num_images=1,
    guidance_scale=7.5,
    num_inference_steps=50,
    seed=None # 推理时允许随机种子
):
    # 确保目录存在
    if not os.path.exists(model_path):
        print(f"错误: 模型路径不存在: {model_path}")
        print("请确保您已经训练了模型或提供了正确的路径")
        return None

    # 改进的设备选择
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"正在加载模型到设备: {device}")
    
    try:
        # 加载微调后的模型
        pipeline = StableDiffusionPipeline.from_pretrained(model_path)
        pipeline = pipeline.to(device)
        
        # 如果有CUDA，尝试启用内存优化
        if device == "cuda":
            try:
                # 尝试启用内存优化
                pipeline.enable_attention_slicing()
                print("已启用 Attention Slicing 优化")
            except Exception as e:
                print(f"注意: 无法启用 Attention Slicing: {e}")
            try:
                # 尝试检测并启用xFormers优化
                pipeline.enable_xformers_memory_efficient_attention()
                print("已启用 xFormers 优化以提高性能")
            except Exception as e:
                print(f"注意: 无法启用 xFormers: {e} (请确保已安装 xformers)")
    
        # 如果未提供标识符但存在标识符文件，则读取它
        if identifier is None and os.path.exists(os.path.join(model_path, "identifier.txt")):
            with open(os.path.join(model_path, "identifier.txt"), "r") as f:
                identifier = f.read().strip()
                print(f"从文件加载标识符: {identifier}")
        
        # 如果未提供提示词，则使用标识符创建
        if identifier is None:
             raise ValueError("错误: 无法找到或确定标识符 (identifier)。请确保模型已训练或手动提供 --identifier 参数。")
        
        # 使用模板构建最终提示词
        prompt = prompt_template.format(identifier)
        
        # 设置生成器以控制随机性
        generator = None
        if seed is not None:
            generator = torch.Generator(device=device).manual_seed(seed)
            print(f"使用种子 {seed} 进行推理")
        else:
             print("使用随机种子进行推理")

        # 生成图像
        print(f"使用提示词生成图像: '{prompt}'")
        
        # 将生成过程包装在 no_grad 中
        with torch.no_grad():
            outputs = pipeline(
                prompt,
                num_images_per_prompt=num_images,
                guidance_scale=guidance_scale,
                num_inference_steps=num_inference_steps,
                generator=generator
            )
        
        # 保存所有生成的图像
        output_dir = os.path.dirname(output_image_path)
        if output_dir:
             os.makedirs(output_dir, exist_ok=True)
        
        saved_paths = []
        if num_images == 1:
            outputs.images[0].save(output_image_path)
            print(f"图像已保存到 {output_image_path}")
            saved_paths.append(output_image_path)
        else:
            base_path, extension = os.path.splitext(output_image_path)
            if not extension:
                 extension = ".png" # 默认扩展名
            for i, image in enumerate(outputs.images):
                path = f"{base_path}_{i}{extension}"
                image.save(path)
                saved_paths.append(path)
            print(f"已保存 {num_images} 张图像到 {output_dir} (以 {base_path}_n{extension} 格式)")
        
        # 清理显存
        del pipeline
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        return outputs.images, saved_paths
    except Exception as e:
        print(f"推理过程中发生错误: {e}")
        import traceback
        traceback.print_exc()
        # 清理显存
        if 'pipeline' in locals():
             del pipeline
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return None, []

## 5. 配置和执行

在此单元格中设置参数，然后运行它以及后续的训练或推理单元格。

In [6]:
# --- 配置参数 ---

# 操作模式 (选择一个)
train_mode = True
infer_mode = False

# 模型设置
use_small_model = True # 是否自动选择小型预训练模型
pretrained_model_name_or_path = None # 如果 use_small_model=False，在此指定模型名称或路径
output_dir = "./dreambooth_output" # 训练后的模型和类别图像保存路径 / 推理时加载模型的路径

# 训练数据设置 (仅训练时需要)
instance_data_dir = "./instance_images" # 包含你的训练图片 (例如 5-10 张 .jpg 或 .png)
class_prompt = "a photo of a dog" # 你的训练对象所属的类别 (例如 "a photo of a dog", "a painting of a landscape")
instance_prompt_template = "a photo of a {} dog" # 实例提示词模板, {} 会被稀有令牌替换

# 训练参数 (仅训练时需要)
learning_rate = 5e-6
max_train_steps = 800
prior_preservation_weight = 1.0 # 先验保留损失权重 (1.0 通常效果好)
prior_generation_samples = 50 # 生成的类别图像数量 (显存小时可减少)
train_text_encoder = True # 是否同时训练文本编码器 (需要更多显存，但通常效果更好)
mixed_precision = "fp16" # 使用混合精度 ('fp16', 'bf16', or 'no')
seed_train = 42

# 推理参数 (仅推理时需要)
prompt_template_infer = "a photo of a {} dog running on the beach" # 推理时使用的提示词模板
output_image_path = "./generated_dreambooth.png" # 生成图像的保存路径
num_images_to_generate = 1 # 要生成的图像数量
guidance_scale = 7.5
num_inference_steps = 50
seed_infer = None # 推理种子 (None 表示随机)

# --- 参数处理 ---
if use_small_model or pretrained_model_name_or_path is None:
    pretrained_model_name_or_path = download_small_model()

print(f"\n将使用的预训练模型: {pretrained_model_name_or_path}")
print(f"输出目录: {output_dir}")

if train_mode:
    print("\n--- 训练模式已启用 ---")
    print(f"实例图像目录: {instance_data_dir}")
    print(f"类别提示词: {class_prompt}")
    print(f"实例提示词模板: {instance_prompt_template}")
    print(f"训练步数: {max_train_steps}")
    print(f"学习率: {learning_rate}")
    print(f"训练文本编码器: {train_text_encoder}")
    print(f"先验保留样本数: {prior_generation_samples}")
    # 检查实例目录是否存在
    if not os.path.exists(instance_data_dir):
        print(f"\n错误: 实例图像目录 '{instance_data_dir}' 不存在！")
        print("请创建该目录并将您的训练图像放入其中。")
        # 可以在这里创建目录，但用户仍需放入图片
        try:
            os.makedirs(instance_data_dir)
            print(f"已创建目录: {instance_data_dir}")
        except OSError as e:
            print(f"创建目录失败: {e}")
        raise FileNotFoundError(f"实例图像目录 '{instance_data_dir}' 不存在或为空。")
    elif not any(f.lower().endswith(('.png', '.jpg', '.jpeg')) for f in os.listdir(instance_data_dir)):
         print(f"\n警告: 实例图像目录 '{instance_data_dir}' 为空或不包含支持的图像文件 (.png, .jpg, .jpeg)。")
         raise FileNotFoundError(f"实例图像目录 '{instance_data_dir}' 为空或不包含图像。")

if infer_mode:
    print("\n--- 推理模式已启用 ---")
    print(f"将从以下路径加载模型: {output_dir}")
    print(f"推理提示词模板: {prompt_template_infer}")
    print(f"输出图像路径: {output_image_path}")

if not train_mode and not infer_mode:
    show_quick_help()
    print("\n请在上方配置单元格中设置 train_mode=True 或 infer_mode=True。")

选择适合低资源设备的小型模型...
已选择模型: CompVis/stable-diffusion-v1-4
此模型与较旧版本的diffusers兼容性更好
模型将在首次使用时自动从Hugging Face下载

将使用的预训练模型: CompVis/stable-diffusion-v1-4
输出目录: ./dreambooth_output

--- 训练模式已启用 ---
实例图像目录: ./instance_images
类别提示词: a photo of a dog
实例提示词模板: a photo of a {} dog
训练步数: 800
学习率: 5e-06
训练文本编码器: True
先验保留样本数: 50


## 6. 执行训练或推理

In [7]:
if train_mode:
    print("\n开始 DreamBooth 训练...")
    # 检查GPU并优化设置
    if torch.cuda.is_available():
        gpu_info = torch.cuda.get_device_name()
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"\n已检测到GPU: {gpu_info} ({gpu_memory:.1f}GB)")
        
        # 根据GPU内存调整优化参数 (示例)
        if gpu_memory < 6 and prior_generation_samples > 30:  # 低端GPU
            print(f"警告: 检测到低显存GPU ({gpu_memory:.1f}GB)。当前先验样本数为 {prior_generation_samples}。")
            print("建议减少先验图像数量 (例如 prior_generation_samples = 30) 以避免内存不足。")
            # 可以选择自动调整或提示用户
            # prior_generation_samples = 30
            # print("已自动调整先验图像数量为 30")
        if gpu_memory < 8 and train_text_encoder:
             print(f"警告: 显存可能不足以同时训练文本编码器。如果遇到 OOM 错误，请尝试设置 train_text_encoder = False。")
    else:
        print("\n警告: 未检测到GPU! 训练将在CPU上运行，这会非常慢!")
        # 在Notebook中通常不强制退出，让用户决定是否继续
        # use_cpu = input("是否继续在CPU上训练? [y/n]: ").lower()
        # if use_cpu != 'y':
        #     print("已取消训练。请在GPU环境下运行。")
        #     # raise RuntimeError("需要GPU进行训练") # 或者抛出错误
            
    try:
        trained_identifier = dreambooth_training(
            pretrained_model_name_or_path=pretrained_model_name_or_path,
            instance_data_dir=instance_data_dir,
            output_dir=output_dir,
            class_prompt=class_prompt,
            instance_prompt_template=instance_prompt_template,
            learning_rate=learning_rate,
            max_train_steps=max_train_steps,
            prior_preservation_weight=prior_preservation_weight,
            prior_generation_samples=prior_generation_samples,
            train_text_encoder=train_text_encoder,
            seed=seed_train,
            mixed_precision=mixed_precision
        )
        print("\n训练完成！")
        print(f"模型已保存到: {output_dir}")
        print(f"学习到的标识符: {trained_identifier}")
        print(f"现在可以将 infer_mode 设置为 True，并使用提示词模板 '{prompt_template_infer.format(trained_identifier)}' 进行推理。")
    except Exception as e:
         print(f"\n训练过程中发生错误: {e}")
         import traceback
         traceback.print_exc()
         print("\n请检查错误信息、参数配置和依赖项安装。")

elif infer_mode:
    print("\n开始 DreamBooth 推理...")
    # 添加CUDA检测
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("\n警告: 未检测到可用的CUDA GPU。将使用CPU运行，这会非常慢!")
        # 在Notebook中通常不强制退出
    else:
        gpu_info = torch.cuda.get_device_name()
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"\n成功检测到GPU: {gpu_info} ({gpu_memory:.1f}GB)")
        print(f"将使用设备: {device}")

    generated_images, saved_paths = inference(
        model_path=output_dir, # 推理时加载训练好的模型
        prompt_template=prompt_template_infer,
        class_prompt=class_prompt, # 用于查找 identifier.txt
        # identifier=None, # 通常让函数自动从文件加载
        output_image_path=output_image_path,
        num_images=num_images_to_generate,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        seed=seed_infer
    )
    
    if generated_images:
        print("\n推理完成！")
        # 可选：在Notebook中显示生成的图像
        from IPython.display import display
        for img_path in saved_paths:
             try:
                 display(Image.open(img_path))
             except Exception as display_e:
                  print(f"无法显示图像 {img_path}: {display_e}")
    else:
        print("\n推理失败。请检查错误信息。")

else:
    print("\n没有选择操作模式。请在配置单元格中设置 train_mode=True 或 infer_mode=True，然后重新运行该单元格和此执行单元格。")


开始 DreamBooth 训练...

已检测到GPU: NVIDIA GeForce RTX 4070 Laptop GPU (8.0GB)
警告: 显存可能不足以同时训练文本编码器。如果遇到 OOM 错误，请尝试设置 train_text_encoder = False。
选中的稀有令牌标识符: 'ier'
实例提示词: 'a photo of a ier dog'
类别提示词: 'a photo of a dog'
加载预训练模型: CompVis/stable-diffusion-v1-4


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 28.56it/s]


KeyboardInterrupt: 